<a href="https://colab.research.google.com/github/shibukawar/stein-importance-sampling-qa/blob/feature%2Fstein-importance-sampling/notebook/example_stein_importance_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

os.chdir("../")

In [ ]:
import time
from collections import defaultdict
from csv import writer

import matplotlib.pyplot as plt
import numpy as np
from dimod.vartypes import Vartype
from dotenv import dotenv_values
from dwave.system.composites import FixedEmbeddingComposite
from dwave.system.samplers import DWaveSampler
from minorminer import find_embedding
from tqdm import tqdm

import stein.config as CONF
from stein.energy import create_hamiltonian
from stein.kernel import KernelType, hamming_kernel
from stein.ksd import DiscreteKSD
from stein.stats import (EmpiricalDistribution, GibbsDistribution,
                         create_empirical_distribution)
from stein.util import (dwave_sampling, flatten_sampleset, pdump, pload,
                        read_from_file, scale_J_h)


In [ ]:
target_temperature = 0.1
annealing_time = 5
PROBLEM = "GSD_8"
gauge_interval = 100

In [ ]:
# convert injected parameters into desired type
dim = 16
target_temperature = float(target_temperature)
annealing_time = float(annealing_time)
gauge_interval = int(gauge_interval)
# const valuable
ENDPOINT = CONF.ENDPOINT

config = dotenv_values(".env")
TOKEN = config['TOKEN']
SOLVER = CONF.SOLVER
PROBLEM_DIR_PATH = f"./problems/{PROBLEM}/"

J, h = read_from_file(PROBLEM_DIR_PATH)
hamiltonian = create_hamiltonian(J, h)
J, h = scale_J_h(J, h, alpha_in=target_temperature)
# Set up dwave sampler
sampler = DWaveSampler(endpoint=ENDPOINT, token=TOKEN, solver=SOLVER)
edges = list(J.keys())

# Define a target distribution

In [ ]:
target_temp_dict = defaultdict(list)

In [ ]:
J, h = read_from_file(PROBLEM_DIR_PATH)
J, h = scale_J_h(J, h, alpha_in=target_temperature)
print(h, J)
edges = np.array(list(J.keys()))
n_sample_list = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
path = os.path.join(
    PROBLEM_DIR_PATH,
    f"samples_{annealing_time}_{target_temperature}.pickle.bz2",
)
if not os.path.exists(path):
    for i in range(5):
        for _ in range(5):
            try:
                embeddings = find_embedding(edges, sampler.edgelist)
                print("Found!!!")
                break
            except ValueError:
                print("Not found!!!")
                pass
        sampler = FixedEmbeddingComposite(sampler, embeddings)
        for n_sample in n_sample_list:
            samples = dwave_sampling(
                sampler,
                h,
                J,
                annealing_time=annealing_time,
                iter_num=int(int(n_sample) / gauge_interval),
                gauge_interval=gauge_interval,
            )
            target_temp_dict[n_sample].append(samples)
    pdump(path, target_temp_dict)
else:
    print(f"{path} already exists.")
    target_temp_dict = pload(path)

In [ ]:
import sqlite3

con = sqlite3.connect("experiments.db")
cur = con.cursor()
cur.execute(
    f"SELECT energy from exact_free_energy WHERE problem = '{PROBLEM}' and beta={target_temperature}"
)
exact_free_energy = cur.fetchone()[0]
cur.execute(
    f"SELECT m2, m4 from exact_m2_m4 WHERE problem = '{PROBLEM}' and beta={target_temperature}"
)
exact_m2, exact_m4 = cur.fetchone()
cur.execute(
    f"SELECT susceptibility from exact_susceptibility WHERE problem = '{PROBLEM}' and beta={target_temperature}"
)
exact_susceptibility = cur.fetchone()[0]
con.commit()
con.close()
exact_squared_mag = exact_free_energy / target_temperature
exact_u4 = 1 - exact_m4 / (3 * exact_m2**2)

In [ ]:
weight_path = os.path.join(
    PROBLEM_DIR_PATH,
    f"weights_{annealing_time}_{target_temperature}.pickle.bz2",
)

In [ ]:
ene_emp_list = []
ene_corr_list = []
u4_emp_list = []
u4_corr_list = []
susceptibility_emp_list = []
susceptibility_corr_list = []
weight_dict = defaultdict(list)
compute_ksd = True
if os.path.exists(weight_path):
    compute_ksd = False
    weight_dict = pload(weight_path)
for n_sample in n_sample_list:
    print(
        f"n_sample: {n_sample}, unique: {[len(sample) for sample in target_temp_dict[n_sample]]}"
    )
    enes_emp = []
    enes_corr = []
    u4_emp = []
    u4_corr = []
    susceptibility_emp = []
    susceptibility_corr = []
    for count, sample in enumerate(target_temp_dict[n_sample]):
        emp_dist = create_empirical_distribution(sample)
        emp_energy = 0
        emp_m2 = 0
        emp_m4 = 0
        emp_susceptibility = 0
        for x in sample.record["sample"]:
            m = sum(x)
            emp_energy += hamiltonian(x) * emp_dist.pmf(x)
            emp_m2 += emp_dist.pmf(x) * m**2
            emp_m4 += emp_dist.pmf(x) * m**4
            emp_susceptibility += emp_dist.pmf(x) * m**2 * target_temperature
        emp_u4 = 1 - emp_m4 / (3 * emp_m2**2)
        enes_emp.append(emp_energy)
        u4_emp.append(emp_u4)
        susceptibility_emp.append(emp_susceptibility)

        weights = None
        if compute_ksd is True:
            trg = GibbsDistribution(hamiltonian, target_temperature, 16, False)
            ksd = DiscreteKSD(
                dim=dim,
                kernel_type=KernelType.Hamming,
                distrib=trg,
                vartype=Vartype.SPIN,
            )
            s = time.time()
            ksd.X_stein_basis = ksd.compute_KP_basis(
                sample.record["sample"], feature_dim=5000
            )
            t = time.time()
            print(f"Elapesed time for kernel computation: {t-s} sec.")
            print("Finish computing kernel.")
            s = time.time()
            ksd.fit_egd(
                sample.record["sample"], n_iter=2000, eta=1e-5, feature_dim=5000
            )
            t = time.time()
            print(f"Elapesed time for fit: {t-s} sec.")
            weights = ksd.weight
            weight_dict[n_sample].append(weights)
        else:
            weights = weight_dict[n_sample][count]
        corr_energy = 0
        corr_m2, corr_m4 = 0, 0
        corr_susceptibility = 0
        for x, w in zip(sample.record["sample"], weights):
            m = sum(x)
            corr_energy += hamiltonian(x) * w
            corr_susceptibility += m**2 * w * target_temperature
            corr_m2 += w * m**2
            corr_m4 += w * m**4
        corr_u4 = 1 - corr_m4 / (3 * corr_m2**2)
        enes_corr.append(corr_energy)
        u4_corr.append(corr_u4)
        susceptibility_corr.append(corr_susceptibility)
        # print(emp_energy, corr_energy)
        # print(emp_u4, corr_u4)
        # print(emp_susceptibility, corr_susceptibility)
    ene_emp_list.append(enes_emp)
    ene_corr_list.append(enes_corr)
    u4_emp_list.append(u4_emp)
    u4_corr_list.append(u4_corr)
    susceptibility_emp_list.append(susceptibility_emp)
    susceptibility_corr_list.append(susceptibility_corr)
if compute_ksd is True:
    pdump(weight_path, weight_dict)

In [ ]:
def plot(emp_list, corr_list, exact, prefix="energy", top=0.6):
    emp_list = np.array(emp_list)
    corr_list = np.array(corr_list)
    res_loss1 = np.abs(emp_list - exact) / np.abs(exact)
    res_loss2 = np.abs(corr_list - exact) / np.abs(exact)
    res1 = np.mean(res_loss1, axis=1)
    res2 = np.mean(res_loss2, axis=1)
    std1 = np.std(res_loss1, axis=1)
    std2 = np.std(res_loss2, axis=1)

    plt.style.use("seaborn-paper")
    _, ax = plt.subplots()
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.errorbar(n_sample_list, res1, std1, marker="*", label="Empirical")
    ax.errorbar(n_sample_list, res2, std2, marker="o", label="Stein")
    ax.set_ylabel("Residual error", fontsize=15)
    ax.set_xlabel("Sample size", fontsize=15)
    ax.set_ylim(bottom=0, top=top)
    ax.set_xticks(n_sample_list, n_sample_list)
    ax.legend(frameon=False, bbox_to_anchor=(1.0, 1.0), fontsize=15)
    plt.savefig(
        os.path.join(
            PROBLEM_DIR_PATH, f"{prefix}_{annealing_time}_{target_temperature}.png"
        ),
        bbox_inches="tight",
    )
    plt.savefig(
        os.path.join(
            PROBLEM_DIR_PATH, f"{prefix}_{annealing_time}_{target_temperature}.pdf"
        ),
        bbox_inches="tight",
    )

In [ ]:
ene_emp_list = np.array(ene_emp_list)
ene_corr_list = np.array(ene_corr_list)
plot(ene_emp_list, ene_corr_list, exact_free_energy, "energy", 1.0)

In [ ]:
plot(
    susceptibility_emp_list,
    susceptibility_corr_list,
    exact_susceptibility,
    "susceptibility",
    0.6,
)

In [ ]:
plot(u4_emp_list, u4_corr_list, exact_u4, "u4", 2.5)

In [ ]:
data = {}
data["energy"] = {}
data["energy"]["exact"] = exact_free_energy
data["energy"]["emp"] = ene_emp_list
data["energy"]["corr"] = ene_corr_list
data["u4"] = {}
data["u4"]["exact"] = exact_u4
data["u4"]["emp"] = u4_emp_list
data["u4"]["corr"] = u4_corr_list
data["sus"] = {}
data["sus"]["exact"] = exact_susceptibility
data["sus"]["emp"] = susceptibility_emp_list
data["sus"]["corr"] = susceptibility_corr_list
path = os.path.join(
    PROBLEM_DIR_PATH, f"raw_data_{annealing_time}_{target_temperature}.pickle.gz2"
)
pdump(path, data)